In [20]:
import glob
import pandas as pd
from shutil import copytree
import json
import pickle
import numpy as np
def random_dna_sequence(length):
    return ''.join(np.random.choice(('A', 'C', 'T', 'G')) for _ in range(length))
import os
fg = pd.read_csv('../../../../../analysis/FreeGenes.csv')

In [7]:
prefix='GGTCTCAA'
suffix='AGAGCTTAGAGACC'

In [37]:
for directory in glob.glob('./../../../data/**'):
    json_file = glob.glob('{}/*.json'.format(directory))
    with open(json_file[0]) as f:
        data = json.load(f)
    with open('./../../../examples/gene.json') as f:
        new_json = json.load(f)
    
    if data['author']['name'] == 'Stanford BIOE80 class 2017':
        new_json['gene_id'] = data['gene_id']
        new_json['part_type'] = data['info']['gene_metadata']['cloning']['part_type']
        new_json['tags'] = ['JCVI-Syn3.0'] + ['cds','moclo']
        new_json['original_sequence'] = data['sequence']['original_sequence']
        new_json['optimized_sequence'] = data['sequence']['optimized_sequence']
        new_json['author'] = [{'name': 'Stanford BIOE80 class 2017', 'email': 'koeng101@gmail.com', 'affiliation': 'Stanford University', 'orcid':''}]
    
        new_json['synthesized_sequence'] = prefix + data['sequence']['optimized_sequence'] + suffix
        
        if len(new_json['synthesized_sequence']) < 300:
            new_json['synthesized_sequence'] = new_json['synthesized_sequence'] + random_dna_sequence(320-len(new_json['synthesized_sequence']))
        
        available = fg[fg['gene_id']==new_json['gene_id']]['sample_available'].tolist()[0]
        seq_type = fg[fg['gene_id']==new_json['gene_id']]['sequencing_type'].tolist()[0]
        
        print('{} - {}'.format(available,seq_type))
        if available == 'Yes' and seq_type == 'NGS':
            new_json['status'] = 'Complete'
        else:
            new_json['status'] = 'Ordered'
        
        #os.makedirs('./../genes/{}'.format(data['gene_id']))
        json_file = './../genes/{}/{}.json'.format(data['gene_id'],data['gene_id'])
        with open(json_file, 'w') as outfile:
            json.dump(new_json, outfile)

No - Sanger
Yes - NGS
No - NGS
BBF10K_000430
Yes - NGS
Yes - NGS
BBF10K_000433
No - nan
No - NGS
Yes - Sanger
Yes - Sanger
No - NGS
No - nan
Yes - Sanger
No - nan
Yes - Sanger
Yes - Sanger
No - nan
Yes - NGS
Yes - Sanger
No - nan
No - nan
No - nan
Yes - Sanger
No - nan
Yes - NGS
Yes - NGS
Yes - NGS
Yes - Sanger
Yes - NGS
Yes - NGS
No - NGS
Yes - Sanger
No - nan
Yes - NGS
BBF10K_000408
No - NGS
BBF10K_000426
Yes - NGS
Yes - NGS
Yes - NGS
No - NGS
No - nan
Yes - NGS
No - nan
Yes - NGS
Yes - NGS
BBF10K_000402
Yes - NGS
Yes - NGS
Yes - NGS
Yes - Sanger
No - nan
Yes - Sanger
No - nan
No - nan
Yes - NGS
Yes - NGS
No - nan
No - NGS
No - NGS
Yes - NGS
Yes - NGS
No - nan
Yes - NGS
Yes - NGS
Yes - NGS
No - nan
Yes - NGS
Yes - NGS
No - NGS
Yes - NGS
Yes - Sanger
No - NGS
Yes - Sanger
No - nan
Yes - NGS
BBF10K_000412
Yes - NGS
BBF10K_000437
No - nan
Yes - NGS
No - nan
No - nan
Yes - NGS
Yes - NGS
No - nan
Yes - Sanger
No - nan
Yes - NGS
No - Sanger
Yes - NGS
Yes - NGS
No - nan
Yes - NGS
Yes - Sang

In [38]:
gene_ids = []
sequences = []
for json_file in glob.glob('./../genes/**/*.json'):
    with open(json_file) as f:
        data = json.load(f)
    if data['status'] == 'Ordered':
        gene_ids.append(data['gene_id'])
        sequences.append(data['synthesized_sequence'])
    
d = {
    "gene_id": gene_ids,
    "sequence": sequences
}
to_twist = pd.DataFrame.from_dict(d)
to_twist.to_csv('./to_twist.csv')

In [40]:
for index,row in pd.read_csv('./to_twist.csv').iterrows():
    if len(row['sequence']) > 5000:
        print(row['gene_id'])

BBF10K_000180


In [41]:
failed = ['BBF10K_000180','BBF10K_000134','BBF10K_000106','BBF10K_000078','BBF10K_000238','BBF10K_000264','BBF10K_000366']

gene_ids = []
sequences = []
for json_file in glob.glob('./../genes/**/*.json'):
    with open(json_file) as f:
        data = json.load(f)
    if data['gene_id'] in failed:
        data['status'] = 'Twist_Failed'
        with open(json_file, 'w') as outfile:
            json.dump(new_json, outfile)